# Error bars

Given the formula that links the differential cross section to the event rate measured by our experiment:

$$
\frac{dR}{d \Omega} = \epsilon_{spectrometer}\left(\theta \right) \cdot \frac{n_{gate}}{T} \cdot n_{c} \cdot \wp(\theta; \lambda', \lambda '') \cdot \frac{d \sigma}{d \Omega}
$$

it is crucial to correctly evaluate the uncertainities of these measurements. 

## Statistical and systematical uncertainties + work flow
First thing first it is fundamental to distinguish between the various sources of errors one can come across during a measurement in a nuclear physics' experiment.
In particular we need to separate the contributions given by $\textit{statistical}$ and $\textit{systematical}$ uncertainties:

- Statistical uncertainities could come from the counting of photons or the errors associated to the fit parameters, a square sum will be used to take account of them.
- Systematical uncertainties arise from various effects, such as the shift of the Compton peak, the errors associated with the geometrical measurements of the apparatus and so on, we'll take account of them with a square sum as well.

By looking at the expression above one could come up with an expression for the uncertainty that looks like:

$$
\left(\frac{\delta R}{R} \right)^2 = \left(\frac{\delta \epsilon_{spectrometer}}{\epsilon_{spectrometer}} \right)^2 + \left(\frac{\delta n_{gate}}{n_{gate}} \right)^2 + \left(\frac{\delta n_c}{n_c} \right)^2 + \left(\frac{\delta \wp}{\wp} \right)^2 + \left(\frac{\delta T}{T} \right)^2
$$

assuming every quantity considered in this expression is independent from one another and has a relatively small error associated to it.

To make sure randomness and bias are not mixed together these two kinds of uncertainties will be combined as:

$$
\sigma_{tot} = \sigma_{statistic} + \sigma_{systematic}
$$

so that one can split the two contributes in:

$$ \left(\frac{\delta Q_{stat}}{Q} \right)^2 = \sum_{X \, \in \, (\epsilon_{spectr.}, n_{gate}, T, n_c, \wp)}  \left(\frac{\delta X_{stat}}{X} \right)^2 $$
$$ \left(\frac{\delta Q_{sist}}{Q} \right)^2 = \sum_{X \, \in \, (\epsilon_{spectr.}, n_{gate}, T, n_c, \wp)}  \left(\frac{\delta X_{sist}}{X} \right)^2 $$


## $n_c$ uncertainty

Given:

$$
n_c = \rho \frac{N_a \cdot Z}{MM}
$$

where: 
- $\rho$ is the density of the scattering target
- $N_a$ is the Avogadro number
- $\text{MM}$ is the molar mass of the scattering target

One can clearly see that all these quantities are known with extreme accuracy, we won't take account of their uncertainties.

## $\epsilon_{spectrometer}$ uncertainty

In this case the value of $\epsilon_{spectrometer}$ is obtained by an interpolation, the error associated to it will then be consider as a statistical uncertainty. Given the formula used to fit the efficiency curve: 

$$
\epsilon_{spectrometer} = A \cdot E^{-B} \cdot \exp(-C \cdot E) + D
$$

where $A,B,C,D$ are all model parameters and $E$ is the energy of the incoming gamma ray. Given that, one can compute the error associated to this quantity as:

$$
\delta \epsilon = \left( \frac{\partial \epsilon}{\partial A} \cdot \delta A \right) \oplus \left( \frac{\partial \epsilon}{\partial B} \cdot \delta B \right) \oplus \left( \frac{\partial \epsilon}{\partial C} \cdot \delta C \right) \oplus \left( \frac{\partial \epsilon}{\partial D} \cdot \delta D \right)
$$

$$
\delta \epsilon = \exp(-E \cdot C) \left( \delta A \cdot E^{-B} - \frac{\delta B \cdot A \cdot B}{E^{B+1}} - \frac{A \cdot C \cdot \delta C }{E^{B}} \right) + \delta D
$$

Parameters from the efficency fit:

  A = 1.55710 ± 2.91779
  B = -0.09831 ± 1.03314
  C = 3.53226 ± 2.39943
  D = 0.10209 ± 0.02355


In [4]:
import numpy as np

# Parametri del fit e le relative incertezze
A = 1.55710
delta_A = 2.91779

B = -0.09831
delta_B = 1.03314

C = 3.53226
delta_C = 2.39943

D = 0.10209
delta_D = 0.02355

def delta_eff(E):
    # Calcolo del termine esponenziale
    exp_term = np.exp(-E * C)
    
    # Calcolo dei singoli contributi all'incertezza
    term1 = delta_A * E**(-B)
    term2 = (delta_B * A * B) / (E**(B + 1))
    term3 = (A * C * delta_C) / (E**B)

    # Formula completa
    delta_epsilon = exp_term * (term1 - term2 - term3) + delta_D
    return delta_epsilon


E_input = 511 
result = delta_eff(E_input)
print(f"\nIncertezza sull'efficienza (δϵ) a E = {E_input:.2f} è {result:.6f}")



Incertezza sull'efficienza (δϵ) a E = 511.00 è 0.023550


## $n_{gate}$ uncertainty

Given: 

$$
n_{gate} = 2 \cdot S \left(t \right) \cdot \text{BR} \cdot \frac{\Delta\Omega}{4\pi} \cdot \epsilon_{gate}(511)
$$

where: 
- $S(t)$ is the activity of the source in Bq
- $\text{BR}$ is the branching ratio of the 511 keV photon
- $\frac{\Delta\Omega}{4\pi}$ is the solid angle
- $\epsilon_{gate}(511)$ is the efficiency of the gate detector for the 511 keV photon

In this case one can assume that the branching ratio (BR) is well known, the uncertainties linked to $S \left(t \right)$ and $\epsilon_{gate}$ are statistical errors and the one associated to $\Delta \Omega$ is systematical.

### $\delta S(t)$ computation
For this uncertainty, an error of 0.5 cm in the distance from the detector and 3 mm in the displacement from the detector axis was considered. The acquisition times were assumed to be error-free, as they were set within the Maestro application. For the number of counts, the uncertainty associated with Poisson statistics was taken into account.
The associated error is then:

$$
\delta S \left( t \right) = 11646,94 \ Bq.
$$

### $\delta \epsilon_{gate}$ computation

For this computation please refer to the previous section.
We impose a 5% error on each point and from the exponential fit we gain:

$$
\delta \epsilon_{gate} = 0.03344 
$$

### $\delta (\Delta \Omega)$ computation

For the uncertainty linked to the measurement of the solid angle covered by the spectrometer one can start from the formula:

$$
\delta \Omega = 2 \pi \cdot \left(1 - \cos (\beta) \right)
$$

where $\beta$ is computed as: 

$$
\beta = \arctan \left( \frac{r_{gate}}{d_{source-gate}} \right).
$$

Both of these two measurements are known with their uncertainty, $r_{gate} = r =  (0.62 \pm 0.05) \, in = (1,575 \pm 0,127) \, cm$ and ${d_{source-gate}} = d =(16.0 \pm 0.1) \, cm$, knowing that one can propagate the errors using the following expression:

$$
\left(\delta (\Delta \Omega) \right)^2 = \left( \frac{\partial (\Delta \Omega)}{\partial r} \delta r\right)^2 + \left( \frac{\partial (\Delta \Omega)}{\partial d} \delta d\right)^2
$$

(IN REALTA' ABBIAMO CHE L'INCERTEZZA SU R DOVREBBE ESSERE STATISTICA PERCHE' FORNITA DAL MANUALE ORTEC QUINDI PRESUMIBILMENTE RPOVENIENTE DA MISURE RIPETUE, MENTRE QUELLA LEGATA ALLA DISTANZA DIREI CHE E' SISTEMATICA PERCHE' VIENE DA UNA SOLA NOSTRA MISURA FATTA A CASO E POI LA DISTANZA GATE SOURCE E' SOGGETTA A FLUTTUAZIONI DOVUTE ALLO SPOSTAMENTO DELL'APPARATO E ROBE VARIE)
and that:

$$
\frac{\partial (\Delta \Omega)}{\partial r} = 2 \pi \sin(\beta) \frac{\partial \beta}{\partial r} = 2 \pi \frac{r \cdot d}{(r^2 + d^2)^{3/2}}
$$

$$
\frac{\partial (\Delta \Omega)}{\partial d} = 2 \pi \sin(\beta) \frac{\partial \beta}{\partial d} = - 2 \pi \frac{r^2}{(r^2 + d^2)^{3/2}}
$$

one can find the uncertainty asssociated to the solid angle using the formula below:

$$
\delta \left(\Delta \Omega \right) = \sqrt{\left(2 \pi \frac{r \cdot d}{(r^2 + d^2)^{3/2}} \cdot \delta r \right)^2 + \left(2 \pi \frac{r^2}{(r^2 + d^2)^{3/2}} \cdot \delta d \right)^2}
$$

obtaining:

$$
\delta(\Delta \Omega) = 3.65 \times 10^{-3} sr
$$





For each of them we get the following values:
- $\sigma_{S \left( t \right)} = 10 491 Bq$ 
- $\sigma_{\epsilon_{gate}} = $
- $\sigma_{\Delta \Omega} = 3.65 \times 10^{-3} sr$

## Final results

Knowing all the information we need, the uncertainty on the rate measurement can be expressed as:

$$
\delta R = \delta_{stat} \pm \delta_{syst} = \left(\delta \epsilon_{spectrometer} \oplus \delta S(t) \oplus \delta \epsilon_{gate} \right) \pm \left( \delta \Delta \Omega\right)
$$



# Energy uncertainties

On the x-axis of our Klein-Nishina plot we put the scattering angles, linked to these quantities there are a few uncertainty factors such as:
- The shift in the position of the Compton peak due to temperature instability and other environmental factors.
- The error associated to the measurement of the scattering angle.
- The uncertainty linked to the spectrometer resolution, which, of course, is not ideal.
- The statistic error derived from the gaussian fit of the Compton peak.

While these first three factors have a systematic nature, the last one can be considered as strictly statistical. Knowing this it is crucial to transform every energy-related quantity into an angle-related quantity, one can do that simply by using the inverted Compton formula shown below:

$$
\theta (E') = \arccos \left[1 + \frac{m_e c^2}{E} \cdot \left( \frac{1}{E'} - \frac{1}{E}\right)\right]
$$

that gives rise to the associated uncertainty formula, which is:

$$
\delta \theta = \left| \frac{d \theta}{dE'}\right| \cdot \delta E' = \left| \frac{m_e c^2/E}{\sqrt{1 - cos^2 \theta} (E')^2}\right| \cdot \delta E' = \left| \frac{m_e c^2}{sin^2 \theta \cdot E \cdot (E')^2}\right| \cdot \delta E'  .
$$







## Peak shift uncertainty

Knowing that the Compton peak undergoes a mean fluctuation called $\Delta E_{peak}$, we used $\delta E_{peak} = \frac{\Delta E_{peak}}{2} $ to quantify our peak energy uncertainty. This can be expressed as:

$$
\delta \theta_{peak} = \left| \frac{m_e c^2}{sin^2 \theta \cdot E \cdot (E')^2}\right| \cdot \delta E_{peak}  .
$$

## Scattering angle uncertainty

This is chosen besed on the instrumental error used to measure $\theta_{scattering}$, and it is called $\delta \theta_{scattering}$.

## Resolution uncertainty

The error due to the resolution of a detector can be expressed by means of the standard deviation obtained from the Compton peak's gaussian fit, so that:

$$
\delta_{resolution} = \sigma_{fit}
$$

Just like the previous case, this quantity needs to be expressed in terms of the scattering angle, this can be computed using:

$$
\delta \theta_{resolution} = \left| \frac{m_e c^2}{sin^2 \theta \cdot E \cdot (E')^2}\right| \cdot \delta_{resolution} 
$$

## Statistical uncertainty

The error derived solely from the fit of the Compton peak can be taken in account with the formula:

$$
\delta_{fit} = \frac{\sigma_{fit}}{\sqrt{N}}
$$

Where N is the number of elements in my Compton peak, here as well one needs to express this error in terms of the scattering angle, the formula used is the same:

$$
\delta \theta_{fit} = \left| \frac{m_e c^2}{sin^2 \theta \cdot E \cdot (E')^2}\right| \cdot \delta_{fit}.
$$

## Final results

One can express the error bars on the x-axis as a sum of statistical and systematical uncertainties following the expression:

$$
\delta \theta = \delta \theta_{stat} \pm \delta \theta_{syst} = (\delta \theta_{fit}) \pm \left(\delta \theta_{peak} \oplus \delta \theta_{scattering} \oplus \delta \theta_{resolution} \right)
$$